操作系统学习——进程管理

#### 操作系统的功能
操作系统的功能主要有九种：
- System initiation 在系统启动并且初始化的时候将对应的操作系统内容装载进内存
- ***管理内存、进程***
- 中断管理（ interupt）
- 定时器（ 一种特殊的中断，CPU会将时间分割成十毫秒的时钟切片，当每个切片处理完之后也会触发中断）
- 文件接口
- 驱动管理
- 进程通信
- 网络通信

##### ***进程及内存管理：*** 

操作系统的进程分为两部分，***外部的用户代码和内部的内核***

用户进程通过系统调用将命令传递给内核态的进程，再通过内核与对应的硬件进行通信（比如磁盘，鼠标，屏幕
***这里记得插张图片

#### 内核架构
操作系统的内核架构分为两种:
- 一种是最为常见的单一内核，特点是在操作系统初始化的时候将所有功能装载到内存空间
- 一种是微内核，特点是只将必要的内容装载在内存空间，剩余的内容放到磁盘空间，多见于一些功能简单，内存较小的操作系统
- linux 所使用的内核架构类似微内核和单一内核的折中产物，部分功能不会在操作系统初始化时被装载而是以模组形式保存在磁盘空间，需要时再装载进内核之中

#### 内存空间布局
- 在内存为4GB的三十二位操作系统中，最低端的地址被分配给了系统的room
- 高位地址，OXC0000……0 到FFFFF……F, 也就是最末尾的1GB被分配给了操作系统的内核
- 中间剩下的地址，被分配给了用户程序。
- Task_size 标志着内核地址和用户地址的分界线（使用task_size 作为分界标记而不是直接规定代码段，为了方便后续修改内存分配不至于出错
在用户空间之中, 从高位到低位地址分别是：
- 栈（stack
- 文件映射
- 堆（heap
- DATA VMA 
- Code VMA\
注意 VMA是虚拟内存描述符，它只存在与逻辑之中，在实际程序运行时VMA会被替换为实际的物理地址
![table](loading_the_data.jpg)

### ****进程控制块/task_struct****
处于内核的一个结构变量，存储了对应的用户进程的堆栈位置和内存影像
- 所有线程共享控制块
- 如果两个进程同时共享一个控制块，他们会使用同一个堆，但是其中一个的栈是从另一个的堆上划分出来的
- 进程控制块连接到本进程的空间描述符再链接到vma描述符
![](task_struct.jpg)

从mm_struct 的映射中就可以看出`p0`, `p1`,`p2` 分别是不同的进程，`p2-1`和`p2-2`是同一进程下的两个线程。
我们可以看出线程虽然有单独的控制块，但是他们所映射到的内存影像是同一个
以及虽然两个线程共用存储，但是他们的堆栈是相对独立的
![](multiple_task_struct.jpg)